In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("../data/processed/output.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247431 entries, 0 to 247430
Data columns (total 5 columns):
ID           247431 non-null object
contenu      247431 non-null object
formation    247431 non-null object
president    247431 non-null object
LABEL        247431 non-null float64
dtypes: float64(1), object(4)
memory usage: 9.4+ MB


In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

data = pd.read_csv("../data/processed/output.csv")

def build_labels():
    print("Building label targets...")
    labels = to_categorical(data["LABEL"], num_classes=4)
    return labels

def build_tokens():
    
    print("Building features...")
    n_most_common_words = 8000
    max_len = 130
    
    print("Building token features...")
    tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(data["contenu"].values)
    
    print("Building sequence features...")
    sequences = tokenizer.texts_to_sequences(data["contenu"].values)
    word_index = tokenizer.word_index
    
    print('Found %s unique tokens.' % len(word_index))
    X = pad_sequences(sequences, maxlen=max_len)
    return X

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import pandas as pd

data = pd.read_csv('../data/processed/output.csv')

def show_loss_and_acc():
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

def show_confusion_matrix(y_test, y_pred):
    y_pred = (y_pred > 0.5)
    conf_mat = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(conf_mat, annot=True, fmt='d',
                xticklabels=data.LABEL.values, yticklabels=data.LABEL.values)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("CONFUSION MATRIX \n", size=16);
    plt.show()

def show_metrics(y_test, y_pred):
    y_pred = (y_pred > 0.5)
    print('\t\t\t\tCLASSIFICATIION METRICS\n')
    print(metrics.classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
from sklearn.model_selection import train_test_split

import pickle

labels = build_labels()
X = build_tokens()

SEED = 0
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=SEED)

print("Train: N = {0} records".format(len(X_train)))
print("Test:  N = {0} records".format(len(X_test)))

with open("../models/saved_model.pickle", "rb") as file: # Use file to refer to the file object
    
    print('Reading models/saved_model.pickle...')
    model_from_pickle = pickle.load(file)

    print('Predicting...')
    y_pred = model_from_pickle.predict(X_test)
    
    show_metrics(y_test, y_pred)
    show_confusion_matrix(y_test, y_pred)

Building label targets...
Building features...
Building token features...
Building sequence features...
Found 382345 unique tokens.
Train: N = 185573 records
Test:  N = 61858 records
Reading models/saved_model.pickle...


/home/pierre/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/pierre/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Predicting...
				CLASSIFICATIION METRICS

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     23732
           1       1.00      0.99      0.99     12762
           2       0.98      0.91      0.94     17025
           3       1.00      0.79      0.88      8339

   micro avg       0.94      0.94      0.94     61858
   macro avg       0.96      0.92      0.94     61858
weighted avg       0.95      0.94      0.94     61858

